In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
class PATH:
    TRAIN = 'data/train.csv'
    TEST = 'data/test.csv'

class CONFIG:
    lr = 0.001
    epoch = 10
    batch_size = 256

In [3]:
df_train = pd.read_csv(PATH.TRAIN)

In [4]:
device = 'cpu'
print(device)

cpu


In [5]:
l = len(df_train)

In [6]:
x = torch.Tensor(df_train.iloc[:, 1:].values).reshape(l, 28, 28) / 256
x = x.unsqueeze(1)
y = torch.Tensor(df_train.iloc[:, 0].values).type(torch.long)

train_x, valid_x = x[:39000], x[39000:]
train_y, valid_y = y[:39000], y[39000:]

In [7]:
class DigitDataset(Dataset):
    
    def __init__(self, x, y = None):
        super(DigitDataset).__init__()
        self.x = x
        self.y = y
        
    def __getitem__(self, idx):
        if self.y == None:
            return self.x[idx]
        return self.x[idx], self.y[idx]
    
    def __len__(self):
        return len(self.x)

In [8]:
trainset = DigitDataset(train_x, train_y)
validset = DigitDataset(valid_x, valid_y)
train_dl = DataLoader(trainset, batch_size = CONFIG.batch_size, shuffle = True)
valid_dl = DataLoader(validset, batch_size = CONFIG.batch_size)

In [9]:
class DigitModel(nn.Module):
    
    def __init__(self):
        super(DigitModel, self).__init__()
        
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, 3),
            nn.Conv2d(32, 8, 3),
            nn.Flatten(),
            nn.Linear(24*24*8, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
        
    def forward(self, x):
        x = self.model(x)
        return x

In [10]:
model = DigitModel().to(device)

In [11]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = CONFIG.lr)

In [ ]:
for e in range(CONFIG.epoch):
    print(f"Training on progress... {e + 1}/{CONFIG.epoch}")
    accuracy = 0
    total = 0
    
    model.train() 
    for x, y in train_dl:
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(x)
        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()
        
    model.eval()
    with torch.no_grad():
        for vx, vy in valid_dl:
            vx = vx.to(device)
            vy = vy.to(device)
            
            prediction = model(vx)
            correct_prediction = prediction.max(dim=1)[1]
            accuracy += (correct_prediction == vy).sum().item()
    print(f"Epoch {e + 1} : Acc {100 * accuracy / 3000}")

Training on progress... 1/10
Epoch 1 : Acc 95.0
Training on progress... 2/10
Epoch 2 : Acc 96.3
Training on progress... 3/10
Epoch 3 : Acc 97.0
Training on progress... 4/10
Epoch 4 : Acc 97.13333333333334
Training on progress... 5/10
Epoch 5 : Acc 97.16666666666667
Training on progress... 6/10
Epoch 6 : Acc 97.23333333333333
Training on progress... 7/10
Epoch 7 : Acc 97.33333333333333
Training on progress... 8/10
Epoch 8 : Acc 97.33333333333333
Training on progress... 9/10
Epoch 9 : Acc 97.53333333333333
Training on progress... 10/10


In [ ]:
df_test = pd.read_csv(PATH.TEST)
l = len(df_test)
test_x = torch.FloatTensor(df_test.values).reshape(l, 28, 28) / 256
test_x = test_x.unsqueeze(1)

In [ ]:
outputs = model(test_x)
_, pred = torch.max(outputs, 1)
pred = pred.cpu()

In [ ]:
for i, img in enumerate(test_x[50:56]):
    plt.subplot(2, 3, i + 1)
    plt.axis('off')
    plt.title(f"Predicted : {pred[50 + i]}")
    plt.imshow(img.squeeze(0))